<a href="https://colab.research.google.com/github/simulate111/Johdatus-teko-lyyn-Introduction-to-Artificial-Intelligence/blob/main/JT_H1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import copy
import time
import queue
import random
from IPython.display import clear_output

In [2]:
pip install imblearn umap-learn onedrivedownloader

In [3]:
from onedrivedownloader import download

link2data = 'https://unioulu-my.sharepoint.com/:u:/g/personal/jukmaatt_univ_yo_oulu_fi/EfLLqEVV7a5AmQRsrbjDBEQBPgyU3vaOCzBiugGq8GvcXQ?e=9AWxtf'

if not os.path.exists('./imgs'):
    print('Downloading data')
    download(link2data, filename='./imgs.zip', unzip=True, unzip_path='./')

# <center>521160P Introduction to artificial intelligence<br><br>Exercise #1<br><br>Search algorithms<br></center>

This exercise deals with depth-first search and breadth-search to solve the river crossing problem and creates heuristic evaluation functions for the 8-puzzle game. **Look return deadlines from moodle**. It is possible the get 4 points from this exercise (2p + 2p)

If you have any questions related to exercises or you face any problems during this exercise please use **moodle forum for programming exercise 1.**

**First fill in your group information (name and student number)**

# Group member information :

* **Member 1 :** `Mohammadreza Akhtari 2401691 `
* **Member 2 :** `...`
* **Member 3 :** `... `

# Task 1. Wolf, sheep, cabbage and shepherd river crossing puzzle with breadth-first and depth first algorithms

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='https://github.com/simulate111/Johdatus-teko-lyyn-Introduction-to-Artificial-Intelligence/blob/main/imgs/rivercrossingpuzzle.png?raw=1' width='700' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Figure 1. Wolf, sheep, cabbage and shepherd river crossing puzzle's initial state.</span>
</div>
<br>

Description: Shepherd is crossing a river with a rowing boat. He must get wolf, sheep and cabbage to other side of the river. The boat is small so it can hold only one item at the time. Shepherd's mission is restricted by following rules:

* Sheep cannot be left alone with the cabbage on the same side, for then the sheep eats the cabbage.

* Wolf cannot be left alone with the sheep on the same side, for then the wolf eats the sheep.

Help shepherd to find right order of moves to get to other side of the river with wolf, sheep and cabbage using **breadth-first** and **depth-first** algorithms. Let's define initial state according to figure 1, where all the objects are on the same side and on the island there is no one.

In [4]:
objects = sorted(['W','S','C','Shp']) # sorted() asures that objects are in alphabetical order
island = sorted([])
mainland = copy.deepcopy(objects)

print(f"Initially on mainland there is {mainland} and on island there is {island}")

Initially on mainland there is ['C', 'S', 'Shp', 'W'] and on island there is []


After this let's make a few helper functions for defining restrictions of the problem

In [5]:
def shepherd_on_island(island):
    """
    On this function it is checked if shepherd is on island
    if shepherd is on island returns True otherwise False
    """
    if "Shp" in island:
        return True
    return False

def objects_on_island(objects, mainland):
    """
    On this function we check which objects are on island when objects on mainland are known.
    Deletes all objects from objects list that are on mainland
    """
    return [obj for obj in objects if obj not in mainland]

def permited_state(objects, mainland):
    """
    On this function it is checked if the planned move is allowed on both mainland and on island
    if move is not allowed function returns False and if move is allowed returns True
    """

    island = objects_on_island(objects, mainland)
    # if shepherd is on island, (wolf and sheep) or (sheep and cabbage) can't be on mainland
    if shepherd_on_island(island):
        if ("W" in mainland and "S" in mainland) or ("S" in mainland and "C" in mainland):
            return False
        return True
    # Using same rules as above but this time shepherd is on mainland
    else:
        if ("W" in island and "S" in island) or ("S" in island and "C" in island):
            return False
        return True

def generate_new_permited_mainlands(objects, mainland):
    "This function will generate new permited states for mainland"
    mainland = copy.deepcopy(mainland)
    new_mainlands = [] # initialize new_mainlands list
    island = objects_on_island(objects, mainland)
    if shepherd_on_island(island):
        island.remove("Shp") # atleast shepherd is moving out of island
        mainland.append("Shp")
        if permited_state(objects, mainland):
            new_mainlands.append(sorted(mainland))
        for obj in island:
            mainland_tmp = mainland + [obj]
            if permited_state(objects, mainland_tmp):
                new_mainlands.append(sorted(mainland_tmp))

    else:
        mainland.remove("Shp") # Atleast shepherd leaves mainland
        island.append("Shp")
        if permited_state(objects, mainland):
            new_mainlands.append(sorted(mainland))
        mainland_tmp = copy.deepcopy(mainland)
        for obj in mainland:
            mainland_tmp.remove(obj)
            if permited_state(objects, mainland_tmp):
                new_mainlands.append(sorted(mainland_tmp))
            mainland_tmp.append(obj)
    return new_mainlands

def print_moves(objects, path):
    """
    This function prints complete path once path is found.
    """
    print('    Mainland' + ' '*23 + 'Boat' +' '*28 + 'Island'+'\n')
    print('*'*110)
    for i, mainland in enumerate(path):
        island = objects_on_island(objects, mainland)
        if shepherd_on_island(island):
            print('     {}'.format(mainland) + ' '*(61-len(str(mainland))) + '{}'.format(island))
            print('*'*110)
            if i<len(path)-1:
                boat = [x for x in island if x not in objects_on_island(objects, path[i+1])]
                print('{}.   '.format(i+1) + ' '*27 + '<-{}'.format(boat))
        else:
            print('     {}'.format(mainland) + ' '*(61-len(str(mainland))) + '{}'.format(island))
            print('*'*110)
            if i<len(path)-1:
                boat = [x for x in mainland if x not in path[i+1]]
                print('{}.   '.format(i+1) + ' '*29 + '{}->'.format(boat))


## Uninformed search

### Depth-first

Implement **depth-first search** using pseudocode bellow. A few steps of the pseudocode is already implmented your job is to fill missing parts.
1. Initialize two empty lists open list and closed list. Open list corresponds to unexplored states and closed list corresponds to already treated states.
2. Define initial state which consists of two other lists mainland and path
3. Add initial state to open list
4. Loo through open list until empty
    1. Pick last element of the open list
    2. if mainland is not in closed_list
        1. Add mainland to closed list
        2. Check if mainland is same as goal
            1. if it is, return path
        3. Generate new mainlands using helper function *generate_new_permited_mainlands(objects, mainland)*
        4. loop through new mainlands one at a time Käy läpi silmukassa uudet mantereet yksi kerrallaan
            1. Solve new path by adding new mainland to current path
            2. add new state to open list which consists of new mainland and new path
5. print error message when open list is empty and no solution was not found.
        

In [6]:
def depth_first(objects, mainland):
    """
    This function solves the river crossing puzzle using depth-first search
    """
    goal = []
    path = [mainland]
    #-------- WRITE HERE YOUR CODE --------
    # 1. Initialize two empty lists: open_list and closed_list
    open_list = []
    closed_list = []

    # 2. Define variable initial_state which is list that consists of two other lists mainland and path
    initial_state = [mainland, path]

    # 3. add initial_state to open_list
    open_list.append(initial_state)

    # 4. Loop through open_list until it is empty (Tip: while loop)
    while open_list:
        # 4.A choose last element of open_list
        mainland, path = open_list.pop()

        # 4.B if variable mainland is not in closed_list
        if mainland not in closed_list:
            # 4.B.a Add variable mainland to closed_list
            closed_list.append(mainland)

            # 4.B.b check if mainland is same as goal
            if mainland == goal:
                # 4.B.b.i if it is return path
                return path

            # 4.B.c Generatating variable new_mainlands using helper function generate_new_permited_mainlands
            new_mainlands = generate_new_permited_mainlands(objects, mainland)

            # 4.B.d Loop through new_mainlands one at a time using loop variable new_mainland
            for new_mainland in new_mainlands:
                # 4.B.d.i Solve variable new_path by adding to list variables path and new_mainland
                new_path = path + [new_mainland]
                # 4.B.d.ii Append to open_list new state which consists of variables new_mainland and new_path
                open_list.append([new_mainland, new_path])

    # 5. print error message when open_list is empty and no solution found
    print("Solution couldn't be found.")
    #-----------------------------------

In [7]:
depth_first_path = depth_first(objects, mainland)
print_moves(objects, depth_first_path)

    Mainland                       Boat                            Island

**************************************************************************************************************
     ['C', 'S', 'Shp', 'W']                                       []
**************************************************************************************************************
1.                                ['S', 'Shp']->
     ['C', 'W']                                                   ['S', 'Shp']
**************************************************************************************************************
2.                              <-['Shp']
     ['C', 'Shp', 'W']                                            ['S']
**************************************************************************************************************
3.                                ['Shp', 'W']->
     ['C']                                                        ['S', 'Shp', 'W']
*************************************

### Breadth-first

In addition implemnt **breadth-first** algorithm. Code is similar to depth-first algorithm only difference is that in pseudocode above on part 4A instead of picking last element of list you pick the first one. Part of the pseudocode is already implemented fill the missing parts

In [8]:
def breadth_first(objects, mainland):
    """
    This function solves river crossing puzzle using breadth-first search.
    Solution is simalr to depth-first search only exception being on pseudocode's part 4.A you pick
    first element of the list instead of last element.
    """
    goal = []
    path = [mainland]
    #-------- WRITE HERE YOUR CODE --------
    # 1. Initialize two empty lists: open_list and closed_list
    open_list = []
    closed_list = []

    # 2. Define variable initial_state which is list that consists of two other lists mainland and path
    initial_state = [mainland, path]

    # 3. add initial_state to open_list
    open_list.append(initial_state)

    # 4. Loop through open_list until it is empty (Tip: while loop)
    while open_list:
        # 4.A choose first element of open_list (Tip: .pop(0) list method)
        mainland, path = open_list.pop(0)

        # 4.B if variable mainland is not in closed_list
        if mainland not in closed_list:
            # 4.B.a Add variable mainland to closed_list
            closed_list.append(mainland)

            # 4.B.b check if mainland is same as goal
            if mainland == goal:
                # 4.B.b.i if it is return path
                return path

            # 4.B.c Generatating variable new_mainlands using helper function generate_new_permited_mainlands
            new_mainlands = generate_new_permited_mainlands(objects, mainland)

            # 4.B.d Loop through new_mainlands one at a time using loop variable new_mainland
            for new_mainland in new_mainlands:
                # 4.B.d.i Solve variable new_path by adding to list variables path and new_mainland
                new_path = path + [new_mainland]
                # 4.B.d.ii Append to open_list new state which consists of variables new_mainland and new_path
                open_list.append([new_mainland, new_path])

    # 5. print error message when open_list is empty and no solution found
    print("Solution couldn't be found.")
    #-----------------------------------

In [9]:
breadth_first_path = breadth_first(objects, mainland)
print_moves(objects, breadth_first_path)

    Mainland                       Boat                            Island

**************************************************************************************************************
     ['C', 'S', 'Shp', 'W']                                       []
**************************************************************************************************************
1.                                ['S', 'Shp']->
     ['C', 'W']                                                   ['S', 'Shp']
**************************************************************************************************************
2.                              <-['Shp']
     ['C', 'Shp', 'W']                                            ['S']
**************************************************************************************************************
3.                                ['C', 'Shp']->
     ['W']                                                        ['C', 'S', 'Shp']
*************************************

**How many steps are required for depth-first and breadth-first algorithms to complete? How they differ from each other?**

` **7 steps.** In th efirst approach or depth first, the first solution is explored into the depth, which is why the first element is carried first. This is while in the second one or breadth first, all possible solutions are investigated at the same level. THe first solution uses also less memory as one branch is explored but in the second approach, all pathes should be examined which uses more memory.`



# Task 2. 8-puzzle game with heuristic estimation functions

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='https://github.com/simulate111/Johdatus-teko-lyyn-Introduction-to-Artificial-Intelligence/blob/main/imgs/eightpuzzle1.png?raw=1' width='400' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Kuva 2. one initial state and goal state for 8-puzzle game.</span>
</div>
<br>

This task creates heuristic evaluation functions for $h(n)$ 8-puzzle game. 8-puzzle consists of eight sliding tile that are numbered from one to eight. Numbered tiles are placed on a 3x3 grid.One square from the grid is empty where you can either horizontally or vertically move one of the tiles next to it. When one of the tiles are moved to an empty tile, a previous place for the tile is left blank.  On 8-puzzle game, the purpose is to reach the target state as few moves from a known initial state. Figure 2 is an 8-puzzle game, an initial state and a target state.

8-puzzle game's search space is so large that with uninformed search algorithms it would take long. Because of that to find solution for game we use informed search algorithm **A\*-search**  

First let's implement a few essential helper functions for game.

In [10]:
def print_state(state):
    """
    Creates printable game table
    """

    printout = ""
    printout += "+---+---+---+\n"
    for i in range(3):
        for j in range(3):
            tile = state[i*3+j]
            printout += "| {} ".format(" " if tile == 0 else tile)
        printout += "|\n"
        printout += "+---+---+---+\n"
    return printout

def solve_permited_moves(location):
    """
    This function solves moves for empty tile
    """

    moves = [1, -1, 3, -3]
    permited_moves = []
    for move in moves:
        if 0 <= location + move < 9:
            if move == 1 and (location == 2 or location == 5 or location == 8):
                continue

            if move == -1 and (location == 0 or location == 3 or location == 6):
                continue

            permited_moves.append(move)
    return permited_moves

def generate_new_permited_states(state):
    empty_tile = state.index(0) # empty tile
    permited_moves = solve_permited_moves(empty_tile)
    new_states = []

    for move in permited_moves:
        new_state = copy.deepcopy(state)
        (new_state[empty_tile + move], new_state[empty_tile]) = (new_state[empty_tile], new_state[empty_tile + move])
        new_states.append(new_state)
    return new_states

def calculate_heuristic_value(state, goal, heuristic_function):
    """
    Calculates value for estimation function depending on chosen heuristic function
    """
    if heuristic_function == "hamming":
        return calculate_hamming_distance(state, goal)
    elif heuristic_function == "city-block":
        return calculate_city_block_distance(state, goal)
    elif heuristic_function == "euclidean":
        return calculate_euclidean_distance(state, goal)
    else:
        print("Options are 'hamming', 'city-block' or 'euclidean'")

def sort_open_list(open_list, f_scores):
    f_scores_tmp = []
    for state, path in open_list:
        f_scores_tmp.append(f_scores[string(state)])
    return [x for y, x in sorted(zip(f_scores_tmp, open_list))]

def string(list_of_strings):
    """Converts list of strings in to string"""
    return "".join(list(map(str, list_of_strings)))

def a_star(initial_state, goal, heuristic_function):
    open_list = []
    closed_list = []
    path = []
    g_scores = {}
    f_scores = {}
    g_scores[string(initial_state)] = 0
    f_scores[string(initial_state)] = g_scores[string(initial_state)] + calculate_heuristic_value(initial_state, goal, heuristic_function)
    state = [initial_state, path]
    open_list.append(state)

    while len(open_list) != 0:
        open_list = sort_open_list(open_list, f_scores)
        state, path = open_list.pop(0)
        if state == goal:
            new_path = path + [state]
            return new_path, closed_list
        closed_list.append(state)
        new_states = generate_new_permited_states(state)
        for new_state in new_states:
            initial_g_score_value = g_scores[string(state)] + 1
            if new_state in closed_list and initial_g_score_value >= g_scores[string(new_state)]:
                continue
            if new_state not in closed_list or initial_g_score_value < g_scores[string(new_state)]:
                new_path = path + [state]
                g_scores[string(new_state)] = initial_g_score_value
                f_scores[string(new_state)] = g_scores[string(new_state)] + calculate_heuristic_value(new_state, goal, heuristic_function)
                if new_state not in open_list:
                    open_list.append([new_state, new_path])

    print("Solution couldn't be found")

def solve_game(initial_state, goal, heuristic_function):
    path, closed_list = a_star(initial_state, goal, heuristic_function)
    for state in path:
        clear_output(wait=True)
        print("Initial state:\n"+ print_state(initial_state))
        print(f"\nSolution with A*-search algorithm using {heuristic_function} distance:\n\n" + print_state(state))
        time.sleep(1)
    print(f"Solution was found in {len(path)-1} moves. And to find those algorithm searched in total {len(closed_list)} states")

## Heuristic estimation functions

The A * search algorithm requires a heuristic estimation function h(n) i.e. a cost estimate, from the state under study to the target state. A typical solution for a game is about 20 moves long and its branching factor i.e. how many moves can be performed on average in one turn is 2.66. This means that a thorough search from the depth of 20 movements goes through $2.66 ^ {20} = 3.15 * 10 ^ {8}$ space. For this reason, it is especially important to find the best possible heuristic function for an 8-puzzle game. Good options for heuristic functions are **Hamming distance**, **city-block distance** (also known as manhattan distance) and **Euclidean distance**.

* Hamming Distance: Counts the number of tiles in the wrong places. A state with few tiles in the wrong places is presumably closer to the target state than, for example, a state with all tiles in the wrong places.

* City-block distance: Calculates the horizontal and vertical absolute distance for each tile from the corresponding target space tile and finally sums them together. $h_{2}(n) = \sum\limits_{i = 1} ^ {8} | a_ {ix} - b_ {ix} | + | a_ {iy} - b_ {iy} |$, where $ a_ {ix} $ is the x-coordinate of the i-th tile of the initial state and $ b_ {iy} $ is the y-coordinate of the i-th tile of the target state.

* Euclidean distance: Calculates the sum of the horizontal and vertical distances squared for each slab from the corresponding target space slab, takes the square root of the sum, and finally sums the square roots together. $h_{3}(n) = \sum\limits_{i=1}^{8} \sqrt{(a_{ix} - b_{ix})^2 + (a_{iy} - b_{iy})^2}$.

The Hamming distance for the situation in Figure 2 is $h_ {1}(n) = 4$, because four of the initial state tiles are in the wrong places compared to the target state. The city-block distance is calculated for Figure 2 as follows: $ h_ {2} (n) = 1 + 0 + 0 + 3 + 2 + 0 + 2 + 0 = 8$. The Euclidean distance, in turn, for the situation in Figure 2 is: $h_ {3}(n) = 1 + 0 + 0 + \sqrt {5} + \sqrt {2} + 0 + 2 + 0 \approx 6.65$. The blank tile in the initial state has no effect on the values of the heuristic functions.

In the A * search algorithm, the depth of the game tree has been used as the path cost estimate $g(n)$. In the algorithm, unexamined states are stored in an open list and processed states in a closed list. Finally, after the algorithm finds a solution, a path is built from the initial state to the target state.

### Euclidean distance heuristic function

Heuristic evaluation functions take the state under study and the target state as arguments. The state under study and the target state are implemented as lists of 9 items, where a value of 0 represents an empty tile. The values in the list run from the upper left corner of the game board to the lower right corner. The initial state in the list in Figure 2 is ***initial state = [7,2,3,1,0,6,5,8,4]*** and the target state is ***target state = [1,2,3,4,5,6,7,8,0]***.

As an example a heuristic Euclidean-distance estimation function is implemented. Your task is to implement the Hamming-distance estimation function and the city-block-distance estimation function and test their performance for the initial state and the target state in Figure 2.

In [11]:
def calculate_euclidean_distance(state, goal):
    """
    This function calculates euclidean distance from state to goal
    """

    # Initialize variable euclidean_value with 0
    euclidean_score = 0
    # Loop through 1 to 8
    for tile_score in range(1, 9):
        # Determine for the state and the target state where in the matrix the value of the tile is (e.g. state_x refers to the tile's horizontal coordinate)
        (state_x, state_y) = (state.index(tile_score) // 3, state.index(tile_score) % 3)
        (goal_x, goal_y) = (goal.index(tile_score) // 3, goal.index(tile_score) % 3)
        # Calculate for each tile squared horizontal and vertical distance and add their sum's square root into variable euclidean_score
        euclidean_score += ((goal_x-state_x)**2 + (goal_y - state_y)**2)**0.5

    return euclidean_score

check that euclidean distance heuristic function returns ~6,65 which was calculated above

In [12]:
# Define initial state and goal according to figure 2
initial_state = [7,2,3,1,0,6,5,8,4]
goal = [1,2,3,4,5,6,7,8,0]

# Check that function returns correct value
print('Euclideam distance for initial state is: {}'.format(calculate_euclidean_distance(initial_state, goal)))

Euclideam distance for initial state is: 6.650281539872885


After this solve game for initial state using euclidean distance as heuristic function

In [13]:
# Solve game using euclidean distance
solve_game(initial_state, goal, 'euclidean')

Initial state:
+---+---+---+
| 7 | 2 | 3 |
+---+---+---+
| 1 |   | 6 |
+---+---+---+
| 5 | 8 | 4 |
+---+---+---+


Solution with A*-search algorithm using euclidean distance:

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 |   |
+---+---+---+

Solution was found in 18 moves. And to find those algorithm searched in total 397 states


### Hamming distance estimation function

Implement heuristic hamming distance function

In [14]:
def calculate_hamming_distance(state, goal):
    """
    This function calculates hamming distance for target state
    """
    # Initialize variable hamming_score with 0
    hamming_score = 0

    # Loop through tile's values from 1 to 8
    for tile_value in range(1, 9):
        # Figure out for state and goal what is the index of a value in given state (e.g state_index is index of tile's value in state parameter). Tip: .index())
        state_index = state.index(tile_value)
        goal_index = goal.index(tile_value)

        # if tile value's index is not equal to goal  value's index
        if state_index != goal_index:
            # increment hamming_score by 1
            hamming_score += 1

    # at the end return hamming_score
    return hamming_score


Confirm that calculate_hamming_distance function returns 4 which was calculated above

In [15]:
# Confirm that right value is returned
print('Hamming-distance for initial state is: {}'.format(calculate_hamming_distance(initial_state, goal)))

Hamming-distance for initial state is: 4


After this solve game using hamming distance

In [16]:
# Solve game with hamming distance heuristic function
solve_game(initial_state, goal, 'hamming')

Initial state:
+---+---+---+
| 7 | 2 | 3 |
+---+---+---+
| 1 |   | 6 |
+---+---+---+
| 5 | 8 | 4 |
+---+---+---+


Solution with A*-search algorithm using hamming distance:

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 |   |
+---+---+---+

Solution was found in 18 moves. And to find those algorithm searched in total 1514 states


### City-block distance estimation function

Implement heuristic city-block distance function

In [17]:
def calculate_city_block_distance(state, goal):
    """
    This function calculates city_block_distance from state to goal
    """
    #-------- YOUR CODE HERE --------
    # Initialize variable city_block_score with 0
    city_block_score = 0

    # Loop through 1 to 8
    for tile_score in range(1, 9):
        # Determine for the state and the goal state where in the matrix the value of the tile is (e.g. state_x is tile's horizontal coordinate and state_y is tile's vertical coordinate)
        (state_x, state_y) = (state.index(tile_score) // 3, state.index(tile_score) % 3)
        (goal_x, goal_y) = (goal.index(tile_score) // 3, goal.index(tile_score) % 3)

        # Calculate for each tile horizontal and vertical absolute distances and add them to variable city_block_score
        city_block_score += abs(goal_x - state_x) + abs(goal_y - state_y)

    # Finally return city_block_score
    return city_block_score
    #-------------------------------------

Confirm that city-block-distance is calculated correctly. According to calculation above it should be 8

In [18]:
# Confirm that function returns correct value
print('City-block distance for initial state is: {}'.format(calculate_city_block_distance(initial_state, goal)))

City-block distance for initial state is: 8


After this let's solve game with city-block distance

In [19]:
# Solve game using city-block distance
solve_game(initial_state, goal, 'city-block')

Initial state:
+---+---+---+
| 7 | 2 | 3 |
+---+---+---+
| 1 |   | 6 |
+---+---+---+
| 5 | 8 | 4 |
+---+---+---+


Solution with A*-search algorithm using city-block distance:

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 |   |
+---+---+---+

Solution was found in 18 moves. And to find those algorithm searched in total 241 states


**Which function was best for this problem based on ammount of explored states?**

`Although all solution were performed based on 18 movements, the city-block distance estimation function explored the least spaces leading to the faster solution.
City-block esplored 241 states.
Euclideanexplored 397 states.
Hamming explored 1514 states.`

# Feedback
Finally answer the following questions

**How much time did you spent on this exercise?**

`Although the main part has been provided, it still requires a lot of time to figure the solution out, especially the city-block. It is better to make it even easier for the first exercise. However, I did like the puzzle solution and the way of illustration while it was solving the problem.`

**Did you face any problems during this exercise? Was notebook material enough for doing this exercise?**

`Your answer here`

**Other feedback related to this exercise**

`Your answer here`

# Returning the exercise

1. Before you return this notebook **make sure that code in notebook works properly** and returns all necessary values by choosing from menu `Kernel -> Restart & Run All`! also make sure you have answer all questions on **bold**
2. Empty all outputs and variables by choosing from menu `Kernel -> Restart & Clear Output`. This step reduces the size of notebook remarkably
3. Rename this notebook in following format **`JT_H1_[student_number(s)].ipynb`** (e.g `JT_H1_1234567.ipynb` or if you have group `JT_H1_1234567_2345678_3456789.ipynb`)
4. Return **only** solved notebook(`file ending with .ipynb`) to moodle programming exercise 1. Everyone must return file to moodle even if you work in a group. **Don't include working directory or other files** when you return the exercise.
5. Finally go answer questions in moodle related to programming exercise 1. Everyone in your group must answer to questions.